* add humidity_feature (boolen) option
* modify degree=1 bug

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

import tkinter
from tkinter import*
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
from tkinter import filedialog
%gui tk

import FFT
import compensation
import module_4D
# import modeling_white_card
import model
import white_card_std

# from modeling_white_card import modeling_white
# from modeling import modeling
import interpolation

In [2]:
def data_loading():
    path = filedialog.askdirectory()
    print(path)
    return path

In [3]:
path = data_loading()

C:/Users/bioinspira/Desktop/Data/Tools/Release code/Regression_model_release_v0.2.1/#20200821154718


In [4]:
#sensor serial number
sensor_number = path.split('/')[-1]
print('sensor_number:', sensor_number)

sensor_number: #20200821154718


# Loading data and FFT transform

In [5]:
# load training data
A = 0
B = 30000
df_n20C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_n20C.csv')[A:B].reset_index(drop=True))
df_n10C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_n10C.csv')[A:B].reset_index(drop=True))
df_0C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_0C.csv')[A:B].reset_index(drop=True))
df_10C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_10C.csv')[A:B].reset_index(drop=True))
train_data_list = [df_n20C, df_n10C, df_0C, df_10C]

# Load testing data
df_test_n20C = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_n20C.csv')[A:B].reset_index(drop=True))
df_test_n10C = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_n10C.csv')[A:B].reset_index(drop=True))
df_test_10C = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_10C.csv')[A:B].reset_index(drop=True))
test_data_list = [df_test_n20C, df_test_n10C, df_test_10C]

In [6]:
print('df_n20C:', df_n20C.shape)
print('df_n10C:', df_n10C.shape)
print('df_0C:', df_0C.shape)
print('df_10C:', df_10C.shape)

print('-------------------------------------')
print('df_test_n20C:', df_test_n20C.shape)
print('df_test_n10C:', df_test_n10C.shape)
print('df_test_10C:', df_test_10C.shape)

df_n20C: (70, 80)
df_n10C: (70, 80)
df_0C: (70, 80)
df_10C: (70, 80)
-------------------------------------
df_test_n20C: (30, 80)
df_test_n10C: (30, 80)
df_test_10C: (30, 80)


# White card model

In [7]:
# white card paremeter setting
name = 'new_test_440nm_degree1'
model_name = Ridge(alpha=10)                    #regression model
channel = '440nm #1'                       #regression channel
degree_white_card = 1                                   #polynimial degree
humidity_step = ['RH40%', 'RH50%', 'RH60%', 'RH70%', 'RH80']
white_card_humidity_feature = False

In [8]:
df_all = pd.concat(train_data_list).reset_index(drop=True)
df_test = pd.concat([df_10C]).reset_index(drop=True)

y_train = df_all[channel]
x_train = df_all[['Temperature', 'Humidity']]
y_test = df_test[channel]
x_test = df_test[['Temperature', 'Humidity']]

os.chdir(path) 
white_card = model.model(name=name,
                         sensor_number=sensor_number,
                         channel=channel,
                         x_train=x_train, 
                         y_train=y_train, 
                         x_test=x_test, 
                         y_test=y_test, 
                         model_name=model_name, 
                         degree=degree_white_card,
                         step=humidity_step,                         
                         ppm=-23,
                         humidity_feature=white_card_humidity_feature)

white_intercept, white_coef = white_card.regression()
white_card.save_white_card_side_coef()
# white_intercept, white_coef = white_card.coef()

In [9]:
# add white card std column
for train_data_comp in train_data_list: 
    white_card_std.create_white_card_std_column(train_data_comp, white_intercept, white_coef, degree_white_card)
for test_data_comp in test_data_list: 
    white_card_std.create_white_card_std_column(test_data_comp, white_intercept, white_coef, degree_white_card) 

In [10]:
# Compensation process
white_card_std = df_10C                       #for constant compensation function
phage_side = 2                                #for compensation
white_card_side = 1                           #for compensation
# comp_channel = "583nm"

for train_data_comp in train_data_list: 
    compensation.compensation_AS7341(train_data_comp, phage_side=phage_side, white_card_side=white_card_side, compChannel=channel)
    
for test_data_comp in test_data_list: 
    compensation.compensation_AS7341(test_data_comp, phage_side=phage_side, white_card_side=white_card_side, compChannel=channel)

## phage model

In [15]:
sensor_humidity_feature = False
model_name = Ridge(alpha=10)                    #regression model
shift = 0                                       #raw data signal shifting
multiple = 1                                    #raw data ppm multiple
train_channel = '440nm #2_mv'                   #training channel
test_channel = '440nm #2_mv'                    #testing channel
degree = 2                                      #polynimial degree
output_modify = 1                               #output multiple

In [16]:
# load training data
dk_n20C = df_n20C.copy().reset_index(drop=True)
dk_n10C = df_n10C.copy().reset_index(drop=True)
dk_0C = df_0C.copy().reset_index(drop=True)  
dk_10C = df_10C.copy().reset_index(drop=True)


# copy testing data
dk_test_n20C = df_test_n20C.copy().reset_index(drop=True)
dk_test_n10C = df_test_n10C.copy().reset_index(drop=True)
dk_test_10C = df_test_10C.copy().reset_index(drop=True)


#setting temperature range
'''dict = {temp. 1 : ([training data1, data2], [blind test data1, data2], white_card_std data)}'''

temp_data_dict = { 

    # n20~10C
    'temp. 1': ([dk_n20C, dk_n10C, dk_0C, dk_10C], [dk_test_n20C]),
    'temp. 2': ([dk_n20C, dk_n10C, dk_0C, dk_10C], [dk_test_n10C]),
    'temp. 3': ([dk_n20C, dk_n10C, dk_0C, dk_10C], [dk_test_10C]),
                  } 
    
# implement temp. range loop
for temp_data, temp_blind_test in temp_data_dict.values():
    

                 
    # plot each model 4D data
#             module_4D.plot_4D_multi_model(temp_data, temp_blind_test, channel_list, sensor_number)

    # concate training data        
    df_all = pd.concat(temp_data).reset_index(drop=True)
    df_test = pd.concat(temp_blind_test).reset_index(drop=True)


    # 取0ppm資料
    dk_all_0ppm = df_all[df_all['ppm']<1].copy()
    dk_test_0ppm = df_test[df_test['ppm']<1].copy()

    # 取10ppm資料
    dk_all_10ppm = df_all[(df_all['ppm']>9)&(df_all['ppm']<11)].copy()
    dk_test_10ppm = df_test[(df_all['ppm']>9)&(df_all['ppm']<11)].copy()

    # 取60ppm資料
    dk_all_60ppm = df_all[(df_all['ppm']>59)].copy()
    dk_test_60ppm = df_test[(df_all['ppm']>59)].copy()

    dk_data_dict = {'0ppm':(dk_all_0ppm, dk_test_0ppm),
                    '10ppm':(dk_all_10ppm, dk_test_10ppm),
                    '60ppm':(dk_all_60ppm, dk_test_60ppm)}

    #training model for 0ppm, 10ppm, 60ppm surface
    surf_coef_list = []                           # store ppm surface coefficient
    for ppm, ppm_data in dk_data_dict.items():

        dk_data, dk_test = ppm_data

        y_train = dk_data[train_channel] + shift 
        x_train = dk_data[['Temperature', 'Humidity']] 

        y_test = dk_test[test_channel] 
        x_test = dk_test[['Temperature', 'Humidity']]

#             channels = train_channel

        ### Ridge regression
#             step = humidity_step
        os.chdir(path)              
        loss_list = []
        loss_each_step_list = []
        sensor = model.model(name=name,
                             sensor_number=sensor_number,
                             channel=train_channel,
                             x_train=x_train, 
                             y_train=y_train, 
                             x_test=x_test, 
                             y_test=y_test, 
                             model_name=model_name, 
                             degree=degree,
                             step=humidity_step,
                             output_modify=output_modify,
                             shift=shift,            
                             multiple=multiple,
                             ppm=ppm,
                             humidity_feature=sensor_humidity_feature)
        intercept, coef = sensor.regression()
        sensor.save_sensor_side_coef()
#         sensor.loss()
#         loss_each_step = sensor.loss_each_step()    
#         sensor.plot()
#         sensor.save_plot()
#         sensor.coef()
        
        #補0調整濕度項次數
        if (sensor_humidity_feature == False)&(degree==1):
            coef = np.append(coef, [0] * degree)

        elif (sensor_humidity_feature == False)&(degree==2):
            coef = np.insert(coef, 1, 0)
            coef = np.append(coef, [0] * degree)

        elif (sensor_humidity_feature == False)&(degree==3):
            coef = np.insert(coef, 1, 0)
            coef = np.insert(coef, 3, 0)
            coef = np.insert(coef, 4, 0)
            coef = np.append(coef, [0] * degree)
            
        coefficient = np.insert(coef, 0, intercept)
        surf_coef_list.append(coefficient) 
   
    test_data = df_test[[test_channel, 'Temperature', 'Humidity', 'ppm']]
    ppm_list = []
    for index, row in test_data.iterrows():
        signal, temperature, humidity, _ = row
        signal_0ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[0], x0=temperature, x1=humidity, degree=degree)
        signal_10ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[1], x0=temperature, x1=humidity, degree=degree)
        signal_60ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[2], x0=temperature, x1=humidity, degree=degree)

#         print(signal_0ppm, signal_10ppm, signal_60ppm)

        ppm = interpolation.interpolation(signal, signal_0ppm, signal_10ppm, signal_60ppm) 
        ppm_list.append(float(ppm))

    # RMSE
    rmse_test = np.sqrt(mean_squared_error(ppm_list, test_data['ppm'])).round(2)

    # Interpolation prediction result    
    predict_test_data = test_data.copy()
    predict_test_data['pred_ppm'] = ppm_list
    temp = int(predict_test_data['Temperature'].mean())
    hum_min = int(predict_test_data['Humidity'].min())
    hum_max = int(predict_test_data['Humidity'].max())
    title = '({}C) ({}%~{}%) ({}) (RMSE={})'.format(temp, hum_min, hum_max, test_channel, rmse_test)
    predict_test_data.iplot(kind='scatter', y=['ppm', 'pred_ppm', 'Humidity', 'Temperature'], title=title)


    # save prediction result
    folderName = '{}'.format(name)
    savePath = os.path.join(os.getcwd(), folderName)
    filename = '({}C)({}%~{}%)({})(shift {})(RMSE={})'.format(temp, hum_min, hum_max, test_channel, shift ,rmse_test)
    plt.plot(predict_test_data['Temperature'], 'r')
    plt.plot(predict_test_data['Humidity'], 'b')
    plt.plot(predict_test_data['ppm'], 'g')
    plt.plot(predict_test_data['pred_ppm'], 'orange')
    plt.ylabel('Temperature/ Humidity/ ppm')
    plt.grid(alpha=0.3)
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),  loc=3, ncol=4, mode="expand")
    plt.title(filename, y=1.15, fontsize=10)
    plt.savefig(savePath + '/' + filename + '.png', dpi=200, bbox_inches='tight')
    plt.close() 
    plt.show()

# Overview 4D data

In [11]:
# 4D plot
training_data = {
             'n20C':df_n20C,
             'n10C':df_n10C,
               '0C':df_0C,
              '10C':df_10C,
                 }

testing_data = {
             'n20C_test':df_test_n20C,
             'n10C_test':df_test_n10C,
              '10C_test':df_test_10C,             
               }

channel_list=['583nm #1', '583nm #1']
module_4D.plot_4D(training_data, testing_data, channel_list, sensor_number)

channel_list=['583nm #2', '583nm #2']
module_4D.plot_4D(training_data, testing_data, channel_list, sensor_number)